<a href="https://colab.research.google.com/github/shlok-py/Pytorch_Tutorial/blob/main/Pytorch_lesson_1_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch tutorial
### Importing dependencies

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Download Fashion MNIST data

In [ ]:
training_data = datasets.FashionMNIST( root = "data",
                                        train=True,
                                      transform=ToTensor(),
                                      download = True)
test_date = datasets.FashionMNIST(root="data",
                                  train = False,
                                  download = True,
                                  transform = ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Load the data using Data Loader

In [ ]:
batch_size = 64
train_data_loader = DataLoader(training_data, batch_size = batch_size)
test_data_loader = DataLoader(test_date, batch_size = batch_size)

### Print the size of loaded data

In [ ]:
for x,y in train_data_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


### Creating models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"with {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_Relu = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_Relu(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

with cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_Relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimising the Model Parameters

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optmizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [ ]:
def train(dataloader, model, loss_fn, optmizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred,y)

        optmizer.zero_grad()
        loss.backward()
        optmizer.step()
        if batch %100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optmizer)
    test(test_data_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.318560  [   64/60000]
loss: 2.286565  [ 6464/60000]
loss: 2.276406  [12864/60000]
loss: 2.254614  [19264/60000]
loss: 2.250027  [25664/60000]
loss: 2.225496  [32064/60000]
loss: 2.216623  [38464/60000]
loss: 2.198077  [44864/60000]
loss: 2.188800  [51264/60000]
loss: 2.137894  [57664/60000]
Test Error: 
 Accuracy: 46.7%, Avg loss: 2.147631 

Epoch 2
-------------------------------
loss: 2.171134  [   64/60000]
loss: 2.143955  [ 6464/60000]
loss: 2.095972  [12864/60000]
loss: 2.095315  [19264/60000]
loss: 2.046201  [25664/60000]
loss: 1.996213  [32064/60000]
loss: 1.997165  [38464/60000]
loss: 1.933789  [44864/60000]
loss: 1.937420  [51264/60000]
loss: 1.835000  [57664/60000]
Test Error: 
 Accuracy: 58.3%, Avg loss: 1.857893 

Epoch 3
-------------------------------
loss: 1.909827  [   64/60000]
loss: 1.856806  [ 6464/60000]
loss: 1.756163  [12864/60000]
loss: 1.778137  [19264/60000]
loss: 1.670488  [25664/60000]
loss: 1.640355  [32064/600

### Save the model as model.pth

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Load the Model

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### Make Predictions

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_date[0][0], test_date[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
